In [7]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

In [8]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins xxxxx


In [66]:
claude = anthropic.Anthropic()

In [17]:
class DietPlannerAssistant:
    def __init__(self, api_key=None):
        """
        Initialize the diet planner with Anthropic client
        
        Args:
            api_key (str, optional): Anthropic API key. 
            If None, uses environment variable ANTHROPIC_API_KEY
        """
        self.client = anthropic.Anthropic(api_key=api_key)
        self.user_profile = {
            "age": None,
            "gender": None,
            "weight": None,
            "height": None,
            "activity_level": None,
            "dietary_goals": None,
            "allergies": [],
            "preferences": []
        }
    
    def collect_user_profile(self, 
        age, gender, weight, height, 
        activity_level, dietary_goals, 
        allergies, preferences
    ):
        """
        Update user profile with collected information
        
        Args:
            Various user profile parameters
        
        Returns:
            Formatted user profile summary
        """
        self.user_profile.update({
            "age": int(age),
            "gender": gender,
            "weight": float(weight),
            "height": float(height),
            "activity_level": activity_level,
            "dietary_goals": dietary_goals,
            "allergies": allergies.split(',') if allergies else [],
            "preferences": preferences.split(',') if preferences else []
        })
        
        return json.dumps(self.user_profile, indent=2)
    
    def clean_history(self, history):
        """
        Clean chat history to remove any extra metadata
        
        Args:
            history (list): Raw chat history
        
        Returns:
            list: Cleaned chat history with only role and content
        """
        cleaned_history = []
        for msg in history:
            # Handle different input formats
            if isinstance(msg, dict):
                # If it's already a dict, extract only role and content
                cleaned_msg = {
                    "role": msg.get("role", ""),
                    "content": msg.get("content", "")
                }
            elif isinstance(msg, list) and len(msg) >= 2:
                # If it's a Gradio-style list, convert to dict
                cleaned_msg = {
                    "role": "user" if len(cleaned_history) % 2 == 0 else "assistant",
                    "content": str(msg[1])
                }
            else:
                # Skip any malformed messages
                continue
            
            # Only add if both role and content are non-empty
            if cleaned_msg["role"] and cleaned_msg["content"]:
                cleaned_history.append(cleaned_msg)
        
        return cleaned_history
    
    def generate_diet_plan(self, message, history):
        """
        Generate personalized diet plan using Claude
        
        Args:
            message (str): User's current message
            history (list): Conversation history
        
        Returns:
            List of messages compatible with Gradio
        """
        # Prepare system prompt with user profile
        system_prompt = f"""
        Jesteś ekspertą ds. żywienia, tworzącym spersonalizowany plan diety.
        Dane użytkownika:
        {json.dumps(self.user_profile, indent=2)}
        
        Wytyczne:
        - Dostosuj plan do profilu użytkownika
        - Podaj szczegółowy, zrównoważony plan żywieniowy
        - Uwzględnij alergie i preferencje
        - Zaproponuj konkretne posiłki i ich wartości odżywcze
        - Podawaj gramatury wszystkich używanych w daniach składników
        - Plan powinien zawierać wszystkie dania na cały tydzień (od poniedziałku do niedzieli)
        - Dieta powinna być zróżnicowana, pełnowartościowa, ma zawierać dużo różnorodnych warzyw, mięs, owoców, nabiałów, węglowodanów i innych, \
        podane preferencje są tylko wskazówką!
        """
        
        # Clean the history
        processed_history = self.clean_history(history)
        
        # Add current message
        processed_history.append({
            "role": "user", 
            "content": message
        })
        
        try:
            # Remove any potential metadata
            clean_messages = [
                {k: msg[k] for k in ['role', 'content']} 
                for msg in processed_history
            ]
            
            output = self.client.messages.create(
                model="claude-3-5-sonnet-20241022",
                messages=clean_messages,
                system=system_prompt,
                max_tokens=1500
            )
            
            # Return full history including new assistant message
            return processed_history + [
                {"role": "assistant", "content": output.content[0].text}
            ]
        
        except Exception as e:
            return processed_history + [
                {"role": "assistant", "content": f"Przepraszam, wystąpił błąd: {str(e)}"}
            ]

In [21]:
# Gradio Interface
def create_diet_planner_interface():
    with gr.Blocks() as demo:
        # User Profile Collection
        gr.Markdown("# 🥗 Personalny Kreator Diety")
        
        with gr.Row():
            age_input = gr.Number(label="Wiek")
            gender_input = gr.Dropdown(
                ["Mężczyzna", "Kobieta"], 
                label="Płeć"
            )
        
        with gr.Row():
            weight_input = gr.Number(label="Waga (kg)")
            height_input = gr.Number(label="Wzrost (cm)")
        
        activity_input = gr.Dropdown(
            ["Niski", "Umiarkowany", "Wysoki"], 
            label="Poziom aktywności"
        )
        
        goals_input = gr.Dropdown(
            ["Utrata wagi", "Wzrost masy mięśniowej", "Utrzymanie wagi"], 
            label="Cel żywieniowy"
        )
        
        allergies_input = gr.Textbox(
            label="Alergie (oddziel przecinkiem)"
        )
        
        preferences_input = gr.Textbox(
            label="Preferencje żywieniowe (oddziel przecinkiem)"
        )
        
        profile_output = gr.JSON(label="Twój Profil")
        
        collect_btn = gr.Button("Zapisz Profil")
        collect_btn.click(
            diet_planner.collect_user_profile, 
            inputs=[
                age_input, gender_input, weight_input, 
                height_input, activity_input, goals_input, 
                allergies_input, preferences_input
            ],
            outputs=profile_output
        )
        
        # Chat Interface
        chatbot = gr.Chatbot(
            placeholder="Witaj! Twój spersonalizowany plan diety jest już prawie gotowy...",
            type="messages"
        )
        
        msg_input = gr.Textbox(label="Zadaj pytanie o dietę")
        submit_btn = gr.Button("Wyślij")
        
        submit_btn.click(
            diet_planner.generate_diet_plan, 
            inputs=[msg_input, chatbot], 
            outputs=[chatbot]
        )
        
        msg_input.submit(
            diet_planner.generate_diet_plan, 
            inputs=[msg_input, chatbot], 
            outputs=[chatbot]
        )
    
    return demo

In [19]:
# Initialize diet planner
diet_planner = DietPlannerAssistant()

In [23]:
# Launch the interface
demo = create_diet_planner_interface()
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://56cd09e9054709f157.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
